# Import Packages

In [17]:

# sfsdds
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scrapper.scrape_games import scrape_games
from scrapper.scrape_games import match_players_to_linestar
from utils.data_loader import load_linestar_data

pd.set_option('display.max_rows', None)


# optimizer
from misc.loader import load_dk_contest, get_player_data, parse_lineup
from pydfs_lineup_optimizer import Player,get_optimizer, Site, Sport, CSVLineupExporter
import re



In [18]:
# Functions


# Post Processing Optimizer results when contest data is available
def evaluate_lineup(lineup, results_lookup):
    """Return a DataFrame with proj/actual/ownership + lineup totals."""
    rows = []
    for p in lineup.players:
        meta = results_lookup.get(p.full_name, {})
        rows.append({
            "Player": p.full_name,
            "Pos": "/".join(p.positions),
            "Team": p.team,
            "Salary": p.salary,
            "Proj_FPTS": p.fppg,                 # or p.projection, depending on lib
            "Actual_FPTS": meta.get("actual_fpts"),
            "%Drafted": meta.get("ownership"),
        })

    df = pd.DataFrame(rows)
    totals = {
        "Proj_Total": lineup.fantasy_points_projection,
        "Actual_Total": df["Actual_FPTS"].sum(skipna=True),
        "%Own": df["%Drafted"].sum(skipna=True),
    }
    return df, totals



def extract_name(s):
    """
    Turn 'Keyonte George(1408151)' -> 'Keyonte George'
    """
    if pd.isna(s):
        return None
    return re.sub(r"\(\d+\)$", "", str(s)).strip()

def lineup_results(row):
    names = [extract_name(row[c]) for c in pos_cols]
    stats = player_stats.reindex(names)  # allow missing players without errors

    actual_total = stats["Actual"].sum(min_count=1)   # total actual FPTS
    total_own    = stats["Own"].sum(min_count=1)      # sum of %Drafted (or change to .mean())

    return pd.Series({
        "Actual_FP": actual_total,
        "Total_Own": total_own,
    })


# Load Data

In [3]:
slate_date = "2025-11-11"

In [4]:
pbp_features = scrape_games(slate_date, force_rescrape=False)
linestar_df = load_linestar_data(slate_date, normalize=False)

2025-11-16 12:18:19,832 - scrapper.scrape_games - INFO - Loading existing PBP features for 2025-11-11


In [5]:
# ---- 1) Try to load contest data for this slate ----
try:
    dk_df = load_dk_contest(slate_date)
    players_df = get_player_data(dk_df)      # the table you showed
except FileNotFoundError:
    dk_df = None
    players_df = None

has_results = players_df is not None and not players_df.empty

# If we have contest data, normalize column names & build lookup
if has_results:
    results_df = (
        players_df
        .rename(columns={
            "Player": "FullName",     # to match optimizer Player.full_name
            "%Drafted": "ownership",
            "FPTS": "actual_fpts",
        })
    )

    results_lookup = (
        results_df
        .set_index("FullName")[["ownership", "actual_fpts"]]
        .to_dict("index")
    )
else:
    results_lookup = {}


In [19]:
# columns that contain players
pos_cols = ["PG", "SG", "SF", "PF", "C", "G", "F", "UTIL"]

# lookup from player name -> ownership + actual FPTS
player_stats = (
    players_df
    .rename(columns={"%Drafted": "Own", "FPTS": "Actual"})
    .set_index("Player")[["Own", "Actual"]]
)


# Process Data

In [6]:
# format pbp table


# get team and salary info
aligned = match_players_to_linestar(pbp_features[['Player']], linestar_df['Name'].tolist())
pbp_features = pbp_features.merge(aligned, on='Player', how='left')

#add team and salary info
merge_cols = linestar_df[['Name', 'Team', 'Salary']]
pbp_features = pbp_features.merge(merge_cols, left_on='Player', right_on='Name', how='left')
pbp_features = pbp_features.drop(columns=['Name'])

#Removing players with NaN salary
pbp_features = pbp_features[pbp_features['Salary'].notna()] 

# Removing players who played less than 5 minutes
pbp_features = pbp_features[pbp_features['Minutes'] >= 10]

# dropping momentum colums
pbp_features = pbp_features.drop(columns=['Momentum'])
 
pbp_features.sort_values(by='ClutchFP', ascending=False)

,Player,FPM,Usage,FP,Minutes,ClutchFP,ClutchRatio,TouchesPerMin,ThreeRate,ScoringFrequency,ScoringConsistency,DominantQuarter,LateGameEmphasis,Substitutions,StarterOrStuffer,Team,Salary
51,Immanuel Quickley,1.453125,0.937500,46.50,32.0,11.25,0.241935,0.656250,0.375000,0.343750,0.245737,3,0.363636,4.0,1,TOR,6400.0
133,Svi Mykhailiuk,1.259259,0.925926,34.00,27.0,10.50,0.308824,0.481481,1.000000,0.333333,0.162573,4,0.555556,3.0,1,UTA,4600.0
55,Isaiah Joe,0.715517,0.758621,20.75,29.0,8.25,0.397590,0.793103,0.833333,0.379310,0.204579,1,0.272727,4.0,0,OKC,4600.0
123,RJ Barrett,0.913793,0.689655,26.50,29.0,7.75,0.292453,0.793103,0.583333,0.241379,0.110276,4,0.428571,4.0,1,TOR,7200.0
60,Jalen Brunson,1.479167,1.333333,53.25,36.0,7.00,0.131455,0.777778,0.375000,0.416667,0.300228,4,0.400000,2.0,1,NYK,8900.0
16,Brooks Barnhizer,0.609375,0.250000,9.75,16.0,6.50,0.666667,0.375000,0.000000,0.187500,0.813008,4,1.000000,2.0,0,OKC,3000.0
128,Santi Aldama,1.091667,0.733333,32.75,30.0,6.00,0.183206,0.533333,0.500000,0.333333,0.108714,1,0.300000,4.0,0,MEM,5600.0
106,Neemias Queta,0.913793,0.413793,26.50,29.0,5.50,0.207547,0.379310,0.250000,0.137931,0.097119,1,0.250000,4.0,0,BOS,4800.0
86,Jusuf Nurkic,0.943548,0.580645,29.25,31.0,5.25,0.179487,0.516129,0.000000,0.258065,0.113921,1,0.375000,3.0,1,UTA,6100.0
66,Jarace Walker,0.670455,0.636364,14.75,22.0,5.00,0.338983,0.909091,0.714286,0.454545,0.261756,3,0.400000,4.0,0,IND,6700.0


In [7]:
# format player pool

# Convert linestar_df to salary_df-style structure
lineup_as_salary = pd.DataFrame({
    "Position": linestar_df["Position"],
    "Name + ID": linestar_df["Name"] + " (" + linestar_df["DFS_ID"].astype(str) + ")",
    "Name": linestar_df["Name"],
    "ID": linestar_df["DFS_ID"],
    # simple stand-in: primary position + /UTIL
    "Roster Position": linestar_df["Position"] + "/UTIL",
    # same salary as in linestar_df
    "Salary": linestar_df["Salary"],
    # simple game info: "TEAM VersusStr"  e.g. "OKC vs GSW"
    "Game Info": linestar_df["Team"] + " " + linestar_df["VersusStr"],
    "TeamAbbrev": linestar_df["Team"],
    # use Projected values as AvgPointsPerGame
    "AvgPointsPerGame": linestar_df["Projected"]
})[[
    "Position",
    "Name + ID",
    "Name",
    "ID",
    "Roster Position",
    "Salary",
    "Game Info",
    "TeamAbbrev",
    "AvgPointsPerGame"
]]
lineup_as_salary['max_exposure']=100
lineup_as_salary['min_exposure']=0
lineup_as_salary.head()
lineup_as_salary.to_csv('players.csv', index = False)

# Setup Optimizer

In [8]:
optimizer = get_optimizer(Site.DRAFTKINGS, Sport.BASKETBALL)
optimizer.load_players_from_csv('players.csv')

# Run/Process Results

In [12]:

has_results = False

for lineup in optimizer.optimize(n=3, max_exposure=0.5):
    print(lineup)                         # existing printout

    if has_results:
        eval_df, totals = evaluate_lineup(lineup, results_lookup)
        #display(eval_df)                  # per-player view with proj / actual / %Drafted
        #print("Projected FP:", totals["Proj_Total"])
        print("Actual FP:   ", totals["Actual_Total"])
        print("%Own:", totals["%Own"])
        print('\n\n\n')


 1. PG      Keyonte George                PG/SG UTA                     38.28          7100.0$   
 2. SG      Payton Pritchard              PG/SG BOS                     31.71          6100.0$   
 3. SF      OG Anunoby                    SF    NYK                     36.31          6500.0$   
 4. PF      Collin Murray-Boyles          C/PF  TOR                     26.36          4600.0$   
 5. C       Andre Drummond                C     PHI                     30.46          5300.0$   
 6. G       Tyrese Maxey                  PG    PHI                     49.5           10300.0$  
 7. F       Kyle Filipowski               C/PF  UTA                     24.5           4000.0$   
 8. UTIL    Jusuf Nurkic                  C     UTA                     33.62          6100.0$   

Fantasy Points 270.74
Salary 50000.00

 1. PG      Keyonte George                PG/SG UTA                     38.28          7100.0$   
 2. SG      Ace Bailey                    SG    UTA                     22.39 

In [23]:
# Optimizer settings

# number of lineups
n = 10 

# max repeating player
max_repeat= 5


# minimum salary for lineup
optimizer.set_min_salary_cap(49000)


# max/min teams
min_teams=4
max_teams=6

# team exposures
#optimizer.set_teams_max_exposures(0.5)  # Set 0.5 exposure for all teams
#optimizer.set_teams_max_exposures(0.5, exposures_by_team={'NYY': 0.8})  # Set 0.5 exposure for all teams except NYY and 0.8 exposure for NYY
#optimizer.set_teams_max_exposures(exposures_by_team={'NYY': 0.8})  # Set 0.5 exposure only for NYY
#optimizer.set_teams_max_exposures(exposures_by_team={'NYY': 0.5, 'NYM': 0.5}, exposure_strategy=AfterEachExposureStrategy)  # Use another exposure strategy



# STACKING


# team stacking

optimizer.add_stack(TeamStack(3))  # stack 3 players
optimizer.add_stack(TeamStack(3, for_teams=['NE', 'BAL', 'KC']))  # stack 3 players from any of specified teams
optimizer.add_stack(TeamStack(3, for_positions=['QB', 'WR', 'TE']))  # stack 3 players with any of specified positions
optimizer.add_stack(TeamStack(3, spacing=2))  # stack 3 players close to each other in range of 2 spots.
optimizer.add_stack(TeamStack(3, max_exposure=0.5))  # stack 3 players from same team with 0.5 exposure for all team stacks
optimizer.add_stack(TeamStack(3, max_exposure=0.5, max_exposure_per_team={'MIA': 0.6}))  # stack 3 players from same team with 0.5 exposure for all team stacks and 0.6 exposure for MIA


# Position Stack

optimizer.add_stack(GameStack(3))  # stack 3 players from the same game
optimizer.add_stack(GameStack(5, min_from_team=2))  # stack 5 players from the same game, 3 from one team and 2 from another



# custom group stack
rodgers_adams_group = PlayersGroup(optimizer.player_pool.get_players('Aaron Rodgers', 'Davante Adams'), max_exposure=0.5)
brees_thomas_group = PlayersGroup(optimizer.player_pool.get_players('Drew Brees', 'Michael Thomas'), max_exposure=0.5)
optimizer.add_stack(Stack([rodgers_adams_group, brees_thomas_group]))



# additional stack




In [13]:


optimizer.set_total_teams(min_teams=min_teams, max_teams=max_teams)
optimizer.set_max_repeating_players(max_repeat)
lineups = list(optimizer.optimize(n))
optimizer.export('result.csv')

In [15]:
lu = pd.read_csv('result.csv', index_col = False)
lu.head()

,PG,SG,SF,PF,C,G,F,UTIL,Budget,FPPG
0,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Tyrese Maxey(1186160),Kyle Filipowski(1403609),Jusuf Nurkic(830642),50000.0,270.74
1,Keyonte George(1408151),Ace Bailey(1519363),Michael Porter Jr.(1061724),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),OG Anunoby(900535),50000.0,270.01
2,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Neemias Queta(1142902),50000.0,269.31
3,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Santi Aldama(1177024),50000.0,269.25
4,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Trendon Watford(1176529),50000.0,268.54


In [ ]:
lu

In [16]:





# add columns to your lineup df
lu[["Actual_FP", "Total_Own"]] = lu.apply(lineup_results, axis=1)

lu.head()


,PG,SG,SF,PF,C,G,F,UTIL,Budget,FPPG,Actual_FP,Total_Own
0,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Tyrese Maxey(1186160),Kyle Filipowski(1403609),Jusuf Nurkic(830642),50000.0,270.74,222.50,211.01
1,Keyonte George(1408151),Ace Bailey(1519363),Michael Porter Jr.(1061724),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),OG Anunoby(900535),50000.0,270.01,253.00,211.33
2,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Neemias Queta(1142902),50000.0,269.31,231.75,183.79
3,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Santi Aldama(1177024),50000.0,269.25,241.00,174.72
4,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Trendon Watford(1176529),50000.0,268.54,216.25,195.42
